In [2]:
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'/opt/spark-2.4.5'

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import udf
from datetime import datetime
from sklearn.cluster import DBSCAN
from collections import Counter
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize']= 20,20
%matplotlib inline
import numpy as np
import pandas as pd
 
 

In [4]:
spark = SparkSession.builder \
    .master("local") \
    .appName("Word Count") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [5]:
s=spark.sparkContext

In [6]:
sql_c=SQLContext(s)
trip= sql_c.read.orc('/FYP/huntersfyp17/dropdatesk=20191108/part*')

In [7]:
trip

DataFrame[tripid: int, passengerid: string, driverid: string, taximodelid: int, pickuplatitude: decimal(15,10), pickuplongitude: decimal(15,10), droplatitude: decimal(15,10), droplongitude: decimal(15,10), pickuptime: timestamp, actualpickuptime: timestamp, actualpickupdatesk: int, dispatchtime: timestamp, droptime: timestamp, comments: string, travelstatus: int, createddate: timestamp, createddatesk: int, driverrating: int, drivercomment: string]

In [44]:
def getDriverTripDF(driverid):
    driver_td_df=trip.select(["tripid","driverid","pickuplatitude","pickuplongitude","droplatitude","droplongitude","pickuptime", "actualpickuptime","droptime"]).where(trip.driverid == driverid ).sort("tripid").collect()
    columns=["tripid","driverid","pickuplatitude","pickuplongitude","droplatitude","droplongitude","pickuptime" ,"actualpickuptime","droptime"]
    df_trip_driver = pd.DataFrame(driver_td_df, columns=columns)
    
    return df_trip_driver

In [45]:
df_trip_driver=getDriverTripDF("4e1020dee5af4da54b0ef441f565971777785152292a1877a2e98a9c6e223ab4")
df_trip_driver

,tripid,driverid,pickuplatitude,pickuplongitude,droplatitude,droplongitude,pickuptime,actualpickuptime,droptime
0,189261598,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.8583200000,79.9281000000,6.8858200000,79.8871000000,2019-11-08 11:52:37,2019-11-08 11:54:35,2019-11-08 12:14:19
1,189267005,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.8890600000,79.8877000000,6.8638400000,79.8872000000,2019-11-08 12:15:15,2019-11-08 12:20:47,2019-11-08 12:32:16
2,189274391,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.8653900000,79.8916000000,6.8076200000,79.9930000000,2019-11-08 12:32:35,2019-11-08 12:42:46,2019-11-08 13:34:09
3,189297318,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.7896900000,79.9914000000,6.8092600000,79.9869000000,2019-11-08 13:36:04,2019-11-08 13:51:49,2019-11-08 13:53:39
4,189308693,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.8162800000,79.9745000000,6.8320100000,79.9641000000,2019-11-08 14:05:29,2019-11-08 14:10:04,2019-11-08 14:16:50
5,189312558,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.8261600000,79.9689000000,6.8283700000,79.9663000000,2019-11-08 14:18:31,2019-11-08 14:32:44,2019-11-08 14:33:43
6,189316347,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.8390800000,79.9650000000,6.9621700000,79.8814000000,2019-11-08 14:33:51,2019-11-08 14:39:25,2019-11-08 15:39:30
7,189327881,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.9649500000,79.8756000000,6.9198100000,79.8689000000,2019-11-08 15:41:57,2019-11-08 15:51:31,2019-11-08 16:29:13
8,189337482,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.9163000000,79.8666000000,6.9864500000,79.9310000000,2019-11-08 16:48:53,2019-11-08 16:53:13,2019-11-08 17:43:30
9,189393948,4e1020dee5af4da54b0ef441f565971777785152292a18...,6.9090700000,79.8544000000,6.9339800000,79.8702000000,2019-11-08 19:30:06,2019-11-08 19:39:55,2019-11-08 20:01:10


In [46]:
for num in df_trip_driver.index:
    df_trip_driver.loc[num,'waitingtime'] =(df_trip_driver['droptime'][num]-df_trip_driver['actualpickuptime'][num]).total_seconds()/60

In [47]:
df_trip_driver.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   tripid            14 non-null     int64         
 1   driverid          14 non-null     object        
 2   pickuplatitude    14 non-null     object        
 3   pickuplongitude   14 non-null     object        
 4   droplatitude      14 non-null     object        
 5   droplongitude     14 non-null     object        
 6   pickuptime        14 non-null     datetime64[ns]
 7   actualpickuptime  14 non-null     datetime64[ns]
 8   droptime          14 non-null     datetime64[ns]
 9   waitingtime       14 non-null     float64       
dtypes: datetime64[ns](3), float64(1), int64(1), object(5)
memory usage: 1.2+ KB


In [54]:
path="driver.txt"
f = open(path, "r")
driverlist=f.readlines()


'0038263cc90dfa55d621a6425e46020136a318d12b974340f518892a6ef690ff\n'

In [65]:
driverlist[0][:-1]

'0038263cc90dfa55d621a6425e46020136a318d12b974340f518892a6ef690ff'